In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


CARGAR DATA SET 

In [4]:
# Cargar datos desde el archivo CSV excluyendo la primera columna (edad)
datos = pd.read_csv('cesaria.csv')
datos = datos.iloc[:, 1:]  # Excluir la primera columna (edad)

# Separar características (X) y etiquetas (y)
X = datos.drop('cesaria', axis=1)
y = datos['cesaria']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar características para mejorar el rendimiento de la red
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
print(datos.head())

   numero  tiempo  presion  cardiaco  cesaria
0       1       0        2         0        0
1       2       0        1         0        1
2       2       1        1         0        0
3       1       0        2         0        0
4       2       0        1         0        1


crear MLP

In [6]:
# Crear el modelo MLP
modelo = Sequential()

# Añadir la capa de entrada y la primera capa oculta
modelo.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Añadir la capa de salida
modelo.add(Dense(units=1, activation='sigmoid'))

# Compilar el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 385 (1.50 KB)
Trainable params: 385 (1.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


ENTRENAR EL MODELO 

In [7]:
# Entrenar el modelo
modelo.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
2/2 [==============================] - 6s 3s/step - loss: 0.6756 - accuracy: 0.5938 - val_loss: 0.5772 - val_accuracy: 0.8750
Epoch 2/50
2/2 [==============================] - 1s 607ms/step - loss: 0.6705 - accuracy: 0.6094 - val_loss: 0.5732 - val_accuracy: 0.8750
Epoch 3/50
2/2 [==============================] - 1s 1s/step - loss: 0.6662 - accuracy: 0.6094 - val_loss: 0.5683 - val_accuracy: 0.8750
Epoch 4/50
2/2 [==============================] - 2s 2s/step - loss: 0.6613 - accuracy: 0.6094 - val_loss: 0.5646 - val_accuracy: 0.8750
Epoch 5/50
2/2 [==============================] - 1s 1s/step - loss: 0.6578 - accuracy: 0.5938 - val_loss: 0.5610 - val_accuracy: 0.8750
Epoch 6/50
2/2 [==============================] - 1s 1s/step - loss: 0.6533 - accuracy: 0.5938 - val_loss: 0.5574 - val_accuracy: 0.8750
Epoch 7/50
2/2 [==============================] - 1s 798ms/step - loss: 0.6502 - accuracy: 0.5938 - val_loss: 0.5540 - val_accuracy: 0.8750
Epoch 8/50
2/2 [===================

In [8]:
# Evaluar el modelo en el conjunto de prueba
puntuacion = modelo.evaluate(X_test, y_test)
print(f"Pérdida: {puntuacion[0]}, Precisión: {puntuacion[1]}")

1/1 [==============================] - 0s 73ms/step - loss: 0.5026 - accuracy: 0.8750
Pérdida: 0.502576470375061, Precisión: 0.875


FUNCIONES INTERFACE GRAFICA 
CARGAR CSV

In [10]:
# Función para cargar un archivo CSV
def cargar_csv():
    global datos, tree
    file_path = filedialog.askopenfilename()
    datos = pd.read_csv(file_path)
    # Mostrar la tabla CSV en el Treeview
    tree.delete(*tree.get_children())
    for index, row in datos.iterrows():
        tree.insert("", index, values=row.tolist())
    # Mostrar mensaje en el área de información
    info_area.insert(tk.END, "Archivo cargado con éxito.\n")

PROCESAR DATOS 

In [12]:
def preprocesar_datos():
    global X_train, X_test, y_train, y_test, scaler, info_area
    datos_limpios = datos.drop('cesaria', axis=1)
    X = datos_limpios.values
    y = datos['cesaria'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Mostrar mensaje en el área de información
    info_area.insert(tk.END, "Datos preprocesados con éxito.\n")

ENTRENAR MODELO 

In [13]:
def entrenar_modelo():
    global modelo, info_area
    modelo = Sequential()
    modelo.add(Dense(units=var_capa_oculta.get(), activation='relu', input_dim=X_train.shape[1]))
    modelo.add(Dense(units=1, activation='sigmoid'))
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    modelo.fit(X_train, y_train, epochs=var_epocas.get(), batch_size=var_tamano_lote.get(), validation_data=(X_test, y_test))
    # Mostrar mensaje en el área de información
    info_area.insert(tk.END, "Modelo entrenado con éxito.\n")

INGRESAR UN VALOR MANUALMENTE 

In [15]:
def predecir_dato():
    try:
        # Obtener valores de entrada manualmente
        edad = float(entry_edad.get())
        embarazos = float(entry_embarazos.get())
        tiempo = float(entry_tiempo.get())
        presion = float(entry_presion.get())
        cardiaco = float(entry_cardiaco.get())

        # Normalizar los datos
        nuevo_dato = scaler.transform([[edad, embarazos, tiempo, presion, cardiaco]])

        # Realizar la predicción
        prediccion = modelo.predict(nuevo_dato)

        # Mostrar el resultado de la predicción
        if prediccion > 0.5:
            resultado.set("Predicción: Cesárea")
        else:
            resultado.set("Predicción: No Cesárea")
    except ValueError:
        resultado.set("Error: Ingresa valores válidos.")


INTERFACE GRAFICA 

In [18]:
# Crear la interfaz gráfica
root = tk.Tk()
root.title("Interfaz Red Neuronal CESAREA")

# Widget para cargar CSV y mostrar tabla
btn_cargar_csv = tk.Button(root, text="Cargar CSV", command=cargar_csv)
btn_cargar_csv.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Widget para mostrar tabla CSV
tree_columns = datos.columns.tolist()
tree = ttk.Treeview(root, columns=tree_columns, show='headings')
for col in tree_columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)
tree.grid(row=1, column=0, padx=10, pady=10, sticky="w")

# Entradas y etiquetas para parámetros
tk.Label(root, text="Épocas:").grid(row=0, column=1, padx=10, pady=10, sticky="w")
var_epocas = tk.StringVar()
entry_epocas = tk.Entry(root, textvariable=var_epocas)
entry_epocas.grid(row=0, column=2, padx=10, pady=10, sticky="w")

tk.Label(root, text="Tamaño del Lote:").grid(row=1, column=1, padx=10, pady=10, sticky="w")
var_tamano_lote = tk.StringVar()
entry_tamano_lote = tk.Entry(root, textvariable=var_tamano_lote)
entry_tamano_lote.grid(row=1, column=2, padx=10, pady=10, sticky="w")

tk.Label(root, text="Neuronas en Capa Oculta:").grid(row=2, column=1, padx=10, pady=10, sticky="w")
var_capa_oculta = tk.StringVar()
entry_capa_oculta = tk.Entry(root, textvariable=var_capa_oculta)
entry_capa_oculta.grid(row=2, column=2, padx=10, pady=10, sticky="w")

# Botón para entrenar el modelo
btn_entrenar = tk.Button(root, text="Entrenar Modelo", command=entrenar_modelo)
btn_entrenar.grid(row=3, column=0, columnspan=3, padx=10, pady=10, sticky="w")

# Entradas y etiquetas para ingresar datos manualmente
tk.Label(root, text="Ingresar Nuevo Dato:").grid(row=4, column=0, columnspan=3, padx=10, pady=10, sticky="w")
tk.Label(root, text="Edad:").grid(row=5, column=0, padx=10, pady=10, sticky="w")
entry_edad = tk.Entry(root)
entry_edad.grid(row=5, column=1, padx=10, pady=10, sticky="w")

tk.Label(root, text="# Embarazos:").grid(row=6, column=0, padx=10, pady=10, sticky="w")
entry_embarazos = tk.Entry(root)
entry_embarazos.grid(row=6, column=1, padx=10, pady=10, sticky="w")

# Botón para predecir y mostrar resultado
btn_predecir = tk.Button(root, text="Predecir", command=predecir_dato)
btn_predecir.grid(row=7, column=0, columnspan=3, padx=10, pady=10, sticky="w")

resultado = tk.StringVar()
lbl_resultado = tk.Label(root, textvariable=resultado)
lbl_resultado.grid(row=8, column=0, columnspan=3, padx=10, pady=10, sticky="w")

# Variable para mostrar información
info_text = tk.StringVar()
lbl_info = tk.Label(root, textvariable=info_text, fg="green")
lbl_info.grid(row=9, column=0, columnspan=3, padx=10, pady=10, sticky="w")

# Iniciar la interfaz gráfica
root.mainloop()

In [17]:
root = tk.Tk()
root.title("Interfaz Red Neuronal CESAREA")

# Marco para cargar CSV y mostrar tabla
frame_csv = tk.Frame(root)
frame_csv.grid(row=0, column=0, padx=10, pady=10, sticky="w")


# Widget para mostrar información
info_area = tk.Text(root, height=10, width=40)
#info_area.pack(padx=10, pady=10)
info_area.grid(row=0, column=0, columnspan=3, padx=10, pady=10)


# Widget para la tabla CSV
tree_frame = tk.Frame(root)
#tree_frame.pack(side=tk.LEFT, padx=10, pady=10)
tree_frame.grid(row=0, column=0,  padx=5, pady=5)

tree_columns = datos.columns.tolist()
tree = ttk.Treeview(tree_frame, columns=tree_columns, show='headings')
for col in tree_columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)
#tree.pack()
tree.grid(row=1, column=0, padx=5, pady=5, sticky="w")

# Botón para cargar archivo CSV
btn_cargar_csv = tk.Button(root, text="Cargar CSV", command=cargar_csv)
#btn_cargar_csv.pack(pady=20)
btn_cargar_csv.grid(row=0, column=0, padx=5, pady=5)

# Botón para preprocesar los datos
btn_preprocesar = tk.Button(root, text="Preprocesar Datos", command=preprocesar_datos)
#btn_preprocesar.pack(pady=10)
btn_preprocesar.grid(row=1, column=1, padx=10, pady=10)

# Etiqueta y entrada para la cantidad de épocas
lbl_epocas = tk.Label(root, text="Épocas:")
#lbl_epocas.pack()
lbl_epocas.grid(row=0, column=0, padx=5, pady=5, sticky="w")
var_epocas = tk.IntVar()
entry_epocas = tk.Entry(root, textvariable=var_epocas)
#entry_epocas.pack()
entry_epocas.grid(row=0, column=1, padx=5, pady=5, sticky="w")

# Etiqueta y entrada para el tamaño del lote
lbl_tamano_lote = tk.Label(root, text="Tamaño del Lote:")
#lbl_tamano_lote.pack()
lbl_tamano_lote.grid(row=1, column=0, padx=5, pady=5, sticky="w")
var_tamano_lote = tk.IntVar()
entry_tamano_lote = tk.Entry(root, textvariable=var_tamano_lote)
#entry_tamano_lote.pack()
entry_tamano_lote.grid(row=1, column=1, padx=5, pady=5, sticky="w")

# Etiqueta y entrada para la cantidad de neuronas en la capa oculta
lbl_capa_oculta = tk.Label(root, text="Neuronas en Capa Oculta:")
#lbl_capa_oculta.pack()
lbl_capa_oculta.grid(row=2, column=0, padx=5, pady=5, sticky="w")
var_capa_oculta = tk.IntVar()
entry_capa_oculta = tk.Entry(root, textvariable=var_capa_oculta)
#entry_capa_oculta.pack()
entry_capa_oculta.grid(row=2, column=1, padx=5, pady=5, sticky="w")

# Botón para entrenar el modelo
btn_entrenar = tk.Button(root, text="Entrenar Modelo", command=entrenar_modelo)
#btn_entrenar.pack(pady=20)
btn_entrenar.grid(row=3, column=0, columnspan=2, pady=10, sticky="w")

frame_manual = tk.Frame(root)
#frame_manual.pack(pady=20)
frame_manual.grid(row=0, column=1, padx=10, pady=10, sticky="w")




# Widget para ingresar datos manualmente
lbl_manual = tk.Label(frame_manual, text="Ingresar Nuevo Dato:")
#lbl_manual.grid(row=0, column=0, columnspan=2)
lbl_manual.grid(row=0, column=0, columnspan=2, pady=5, sticky="w")

lbl_edad = tk.Label(frame_manual, text="Edad:")
lbl_edad.grid(row=1, column=0, padx=5, pady=5, sticky="w")
entry_edad = tk.Entry(frame_manual)
entry_edad.grid(row=1, column=1, padx=5, pady=5, sticky="w")

lbl_embarazos = tk.Label(frame_manual, text="# Embarazos:")
lbl_embarazos.grid(row=2, column=0, padx=5, pady=5, sticky="w")
entry_embarazos = tk.Entry(frame_manual)
entry_embarazos.grid(row=2, column=1, padx=5, pady=5, sticky="w")

lbl_tiempo = tk.Label(frame_manual, text="Tiempo:")
lbl_tiempo.grid(row=3, column=0, padx=5, pady=5, sticky="w")
entry_tiempo = tk.Entry(frame_manual)
entry_tiempo.grid(row=3, column=1, padx=5, pady=5, sticky="w")

lbl_presion = tk.Label(frame_manual, text="Presión:")
lbl_presion.grid(row=4, column=0, padx=5, pady=5, sticky="w")
entry_presion = tk.Entry(frame_manual)
entry_presion.grid(row=4, column=1, padx=5, pady=5, sticky="w")

lbl_cardiaco = tk.Label(frame_manual, text="Cardíaco:")
lbl_cardiaco.grid(row=5, column=0, padx=5, pady=5, sticky="w")
entry_cardiaco = tk.Entry(frame_manual)
entry_cardiaco.grid(row=5, column=0, padx=5, pady=5, sticky="w")

btn_predecir = tk.Button(frame_manual, text="Predecir", command=predecir_dato)
btn_predecir.grid(row=6, column=0, columnspan=2)

resultado = tk.StringVar()
lbl_resultado = tk.Label(frame_manual, textvariable=resultado)
lbl_resultado.grid(row=7, column=0, padx=5, pady=5, sticky="w")

# Variable para mostrar información
info_text = tk.StringVar()
lbl_info = tk.Label(root, textvariable=info_text, fg="green")
#lbl_info.pack()
lbl_info.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")

# Iniciar la interfaz gráfica
root.mainloop()